In [1]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.jmt.*
import jline.solvers.mva.*
import jline.util.matrix.*

In [5]:
val model = Network("model")

// Nodes
val delay = Delay(model, "Delay")
val fork1 = Fork(model, "Fork1")
fork1.setTasksPerLink(1)
val fork11 = Fork(model, "Fork1_1")
fork11.setTasksPerLink(2)
val join1 = Join(model, "Join1", fork1)
val join11 = Join(model, "Join1_1", fork11)
val queue1 = Queue(model, "Queue1", SchedStrategy.PS)
val queue2 = Queue(model, "Queue2", SchedStrategy.PS)

// Classes
val jobclass1 = ClosedClass(model, "class1", 5, delay, 0)
val jobclass2 = ClosedClass(model, "class2", 2, delay, 0)

In [6]:
// Service times
delay.setService(jobclass1, Exp(0.25))
queue1.setService(jobclass1, Exp(1.0))
queue2.setService(jobclass1, Exp(0.75))

delay.setService(jobclass2, Exp(0.25))
queue1.setService(jobclass2, Exp(2.0))
queue2.setService(jobclass2, Exp(2.0))

In [7]:
// Routing
val P = model.initRoutingMatrix()

// Class 1 routing - simple fork-join
P.set(jobclass1, jobclass1, delay, fork1, 1.0)
P.set(jobclass1, jobclass1, fork1, queue1, 1.0)
P.set(jobclass1, jobclass1, fork1, queue2, 1.0)
P.set(jobclass1, jobclass1, queue1, join1, 1.0)
P.set(jobclass1, jobclass1, queue2, join1, 1.0)
P.set(jobclass1, jobclass1, join1, delay, 1.0)

// Class 2 routing - nested fork-join
P.set(jobclass2, jobclass2, delay, fork11, 1.0)
P.set(jobclass2, jobclass2, fork11, fork1, 1.0)
P.set(jobclass2, jobclass2, fork1, queue1, 1.0)
P.set(jobclass2, jobclass2, fork1, queue2, 1.0)
P.set(jobclass2, jobclass2, queue1, join1, 1.0)
P.set(jobclass2, jobclass2, queue2, join1, 1.0)
P.set(jobclass2, jobclass2, join1, join11, 1.0)
P.set(jobclass2, jobclass2, join11, delay, 1.0)

model.link(P)

In [8]:
// Solve with different solversval solvers = arrayOf(    JMT(model, "seed", 23000),    MVA(model))// Get average tablesfor (solver in solvers) {        try {        val avgTable = solver.avgTable        avgTable.print()    } catch (e: Exception) {        println("Error: ${e.message}")    }}


SOLVER: JMT
JMT Model: /tmp/workspace/jsim/15515341752778076833/jmodel.jsim
JMT [method: default, lang: java, env: 17.0.15] completed in 1.660021s.
Station  JobClass  QLen        Util        RespT       ResidT      ArvR        Tput        
-------------------------------------------------------------------------------------------
Delay    class1    1.99342     1.99342     3.96607     3.96607     0.50404     0.50379     
Delay    class2    1.01248     1.01248     3.98312     3.98312     0.25569     0.25574     
Join1    class1    2.19429     0           2.22187     2.22187     1.01502     0.50404     
Join1    class2    1.00619     0           0.99526     0.99526     1.02594     0.50666     
Join1_1  class2    0.51907     0           1.01999     1.01999     0.50666     0.25395     
Queue1   class1    1.40339     0.50771     2.69398     1.34699     0.50379     0.50150     
Queue1   class2    0.85773     0.25614     1.66238     0.83119     0.50686     0.51104     
Queue2   class1    2.37